In [3]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
df = pd.read_csv("/content/drive/MyDrive/quora/train.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df[['question1','question2']][df['is_duplicate']==1].values[0]

array(['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
       "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"],
      dtype=object)

In [6]:
df[['question1','question2']][df['is_duplicate']==0].values[0]

array(['What is the step by step guide to invest in share market in india?',
       'What is the step by step guide to invest in share market?'],
      dtype=object)

In [ ]:
sample_df = df.sample(30000)

In [ ]:
sample_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [ ]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
     "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have", 
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    #lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word, tag in pos_tag(q.split()):
      wntag = tag[0].lower()
      wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
      if not wntag: 
        lemma = word
      else:
        lemma = lemmatizer.lemmatize(word, wntag)
      lemmas.append(lemma)    
    return " ".join(lemmas)

In [ ]:
preprocess("I have three visions for India. In 3000 years of our history, people from all over ")

'i have three vision for india in 3k year of our history people from all over'

In [ ]:
sample_df['question1'] = sample_df['question1'].apply(preprocess)
sample_df['question2'] = sample_df['question2'].apply(preprocess)

In [ ]:
#length of each question
sample_df['q1_len'] = sample_df['question1'].str.len() 
sample_df['q2_len'] = sample_df['question2'].str.len()

In [ ]:
#no of words in each question
sample_df['q1_num_words'] = sample_df['question1'].apply(lambda row: len(row.split(" ")))
sample_df['q2_num_words'] = sample_df['question2'].apply(lambda row: len(row.split(" ")))

In [ ]:
#count of common words counts in both question 
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)
sample_df['word_common'] = sample_df.apply(common_words, axis=1)

In [ ]:
#sum of total words in both questions
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))
sample_df['total_words'] = sample_df.apply(total_words, axis=1)

In [ ]:
#percentages of words share between both the questions
sample_df['word_share'] = round(sample_df['word_common']/sample_df['total_words'],2)

In [ ]:
sample_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,total_words,word_share
69644,69644,120148,69901,what be a good song i can use to lyric prank m...,what be some good song to lyric prank your mom,0,58,46,14,10,7,24,0.29
377242,377242,263566,94560,can you see who visit your instagram last,be there a way to see who view my instagram page,1,41,48,8,11,3,19,0.16
399079,399079,532328,172299,what be the advantage of vim over other text e...,what be the point to learn and use vim by just...,0,51,94,10,21,6,29,0.21
98586,98586,163809,163810,what be the best material to prepare for ssc c...,what be the best book to prepare for ssc cgl t...,1,60,51,13,12,11,25,0.44
123391,123391,199510,199511,how do the economy of india work,how do india s economy work,1,32,27,7,6,5,13,0.38


In [ ]:
# Advanced Features
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [ ]:
token_features = sample_df.apply(fetch_token_features, axis=1)

sample_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
sample_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
sample_df["csc_min"]       = list(map(lambda x: x[2], token_features))
sample_df["csc_max"]       = list(map(lambda x: x[3], token_features))
sample_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
sample_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
sample_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
sample_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [ ]:
sample_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,total_words,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
69644,69644,120148,69901,what be a good song i can use to lyric prank m...,what be some good song to lyric prank your mom,0,58,46,14,10,...,24,0.29,0.799984,0.571420,0.599988,0.428565,0.699993,0.499996,0.0,1.0
377242,377242,263566,94560,can you see who visit your instagram last,be there a way to see who view my instagram page,1,41,48,8,11,...,19,0.16,0.499988,0.399992,0.249994,0.166664,0.374995,0.272725,0.0,0.0
399079,399079,532328,172299,what be the advantage of vim over other text e...,what be the point to learn and use vim by just...,0,51,94,10,21,...,29,0.21,0.499988,0.199998,0.666656,0.444440,0.599994,0.285713,1.0,1.0
98586,98586,163809,163810,what be the best material to prepare for ssc c...,what be the best book to prepare for ssc cgl t...,1,60,51,13,12,...,25,0.44,0.857131,0.749991,0.999980,0.999980,0.916659,0.846147,0.0,1.0
123391,123391,199510,199511,how do the economy of india work,how do india s economy work,1,32,27,7,6,...,13,0.38,0.999967,0.999967,0.666644,0.499988,0.833319,0.714276,1.0,1.0


In [ ]:
pip install distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16258 sha256=ed9febb3266beee6598217a632bba5509405d6d235f44ddf702adddc5c8b109e
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [ ]:
import distance

def fetch_length_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [ ]:
length_features = sample_df.apply(fetch_length_features, axis=1)

sample_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
sample_df['mean_len'] = list(map(lambda x: x[1], length_features))
sample_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))

In [ ]:
sample_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
69644,69644,120148,69901,what be a good song i can use to lyric prank m...,what be some good song to lyric prank your mom,0,58,46,14,10,...,0.571420,0.599988,0.428565,0.699993,0.499996,0.0,1.0,4.0,12.0,0.340426
377242,377242,263566,94560,can you see who visit your instagram last,be there a way to see who view my instagram page,1,41,48,8,11,...,0.399992,0.249994,0.166664,0.374995,0.272725,0.0,0.0,3.0,9.5,0.261905
399079,399079,532328,172299,what be the advantage of vim over other text e...,what be the point to learn and use vim by just...,0,51,94,10,21,...,0.199998,0.666656,0.444440,0.599994,0.285713,1.0,1.0,11.0,15.5,0.230769
98586,98586,163809,163810,what be the best material to prepare for ssc c...,what be the best book to prepare for ssc cgl t...,1,60,51,13,12,...,0.749991,0.999980,0.999980,0.916659,0.846147,0.0,1.0,1.0,12.5,0.576923
123391,123391,199510,199511,how do the economy of india work,how do india s economy work,1,32,27,7,6,...,0.999967,0.666644,0.499988,0.833319,0.714276,1.0,1.0,1.0,6.5,0.321429


In [ ]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [ ]:
fuzzy_features = sample_df.apply(fetch_fuzzy_features, axis=1)

# Creating sample feature columns for fuzzy features
sample_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
sample_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
sample_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
sample_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [ ]:
sample_df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'q1_len', 'q2_len', 'q1_num_words', 'q2_num_words', 'word_common',
       'total_words', 'word_share', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max',
       'ctc_min', 'ctc_max', 'last_word_eq', 'first_word_eq', 'abs_len_diff',
       'mean_len', 'longest_substr_ratio', 'fuzz_ratio', 'fuzz_partial_ratio',
       'token_sort_ratio', 'token_set_ratio'],
      dtype='object')

In [ ]:
sample_df.to_csv("engineered_df.csv")